<a href="https://colab.research.google.com/drive/1qWyYKcaXtqF0ewSxpY7fS0HkEZ6DdyZE?usp=sharing" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

In [ ]:
!pip install -q unsloth "torch>=2.1" transformers accelerate peft datasets bitsandbytes



In [ ]:
!pip install -q trl

In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from peft import LoraConfig

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
    use_rslora=False,
)

==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.4.1 patched 22 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
dataset = load_dataset("tatsu-lab/alpaca", split="train[:10%]")  # Use 10% for quicker demo

README.md:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


(…)-00000-of-00001-a09b74b3ef9c3b56.parquet:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
def format_chatml(example):
    user_prompt = example['instruction']
    if example.get('input'):
        user_prompt += f"\n{example['input']}"

    return {
        "prompt": f"<|im_start|>user\n{user_prompt}\n<|im_end|>\n<|im_start|>assistant\n",
        "output": f"{example['output']}<|im_end|>"
    }

dataset = dataset.map(format_chatml)

Map:   0%|          | 0/5200 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import Dataset

class SupervisedDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        prompt = self.dataset[idx]["prompt"]
        output = self.dataset[idx]["output"]
        full_prompt = prompt + output

        tokenized = self.tokenizer(full_prompt, truncation=True, padding="max_length", max_length=2048, return_tensors="pt")
        input_ids = tokenized.input_ids.squeeze(0)
        labels = input_ids.clone()
        return {"input_ids": input_ids, "labels": labels}

train_dataset = SupervisedDataset(dataset, tokenizer)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./tinyllama-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    warmup_steps=10,
    max_steps=100,
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    logging_steps=10,
    save_strategy="no",
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
    tokenizer=tokenizer,
    dataset_text_field="text",
    packing=False,
)

trainer.train()



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,200 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 4,505,600/4,000,000,000 (0.11% trained)


Step,Training Loss
10,2.315800
20,1.567400
30,1.295800
40,1.050000
50,1.117200
60,1.251700
70,1.000200
80,0.972900
90,1.025600
100,1.007300


TrainOutput(global_step=100, training_loss=1.2603971576690673, metrics={'train_runtime': 100.0285, 'train_samples_per_second': 1.0, 'train_steps_per_second': 1.0, 'total_flos': 1276745298739200.0, 'train_loss': 1.2603971576690673})

In [ ]:
model.save_pretrained("./tinyllama-finetuned-model")
tokenizer.save_pretrained("./tinyllama-finetuned-model")

('./tinyllama-finetuned-model/tokenizer_config.json',
 './tinyllama-finetuned-model/special_tokens_map.json',
 './tinyllama-finetuned-model/tokenizer.model',
 './tinyllama-finetuned-model/added_tokens.json',
 './tinyllama-finetuned-model/tokenizer.json')

In [ ]:
def chat_response(prompt, max_new_tokens=200):
    chat_prompt = f"<|im_start|>user\n{prompt}\n<|im_end|>\n<|im_start|>assistant\n"
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    start = decoded.find("<|im_start|>assistant\n")
    if start != -1:
        response = decoded[start + len("<|im_start|>assistant\n"):].strip()
    else:
        response = decoded.strip()
    response = response.split("<|im_end|>")[0].strip()
    return response

# Test the model
test_prompts = [
    "Explain photosynthesis in simple terms.",
    "What is the formula to find the area of a triangle?",
    "Give me 5 tips to stay productive while working from home.",
    "Write a short poem about the stars.",
    "What is the meaning of life according to different philosophers?",
    "How can I manage my time better as a student?",
    "Explain the difference between RAM and ROM.",
    "Suggest some daily habits to improve mental well-being.",
]

for prompt in test_prompts:
    print(f"Prompt: {prompt}")
    print(f"Response: {chat_response(prompt)}")
    print("="*100)

Prompt: Explain photosynthesis in simple terms.
Response: Photosynthesis is the process by which plants, algae, and some bacteria convert sunlight into energy. It is a complex process that involves the conversion of carbon dioxide into glucose (a type of sugar) through the use of chlorophyll. The process is essential for the survival of many organisms, as it provides the energy needed for growth, reproduction, and survival.
Prompt: What is the formula to find the area of a triangle?
Response: The formula to find the area of a triangle is A = 0.5(b x h).
Prompt: Give me 5 tips to stay productive while working from home.
Response: 1. Stay organized: Use a planner or calendar to keep track of your tasks and deadlines.
2. Set boundaries: Set specific times for work and personal time.
3. Take breaks: Take regular breaks to stretch, walk, or do something else to avoid burnout.
4. Stay connected: Use technology to stay connected with colleagues and clients.
5. Prioritize tasks: Prioritize tas